In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [127]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataset import TensorDataset
from torch_geometric.data import DataLoader
from torch_geometric.data import Data
from torch_geometric.utils.convert import to_networkx
import torch.nn as nn

import networkx as nx # for visualizing graphs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from functions.load_data import MarielDataset, edges

##### Encoder Procedure
1. Create an embedding of the node features (H) using an MLP
2. Create a message to pass through the edges of the graph using the embedded node features (H) and another MLP
3. Aggregate the messages created in Step 2 for each node to update node features
4. Pass the updated node features through another MLP to get the "Pre-posterior"; the posterior then becomes the softmax of the pre-posterior

# Load data

In [117]:
data = MarielDataset(seq_len=10, n_joints=53, reduced_joints=False)
dataloader = DataLoader(data, batch_size=2, shuffle=False)
print("\nGenerated {:,} sequences of shape: {}".format(len(data), data[0]))

Original numpy dataset contains 38,309 timesteps of 53 joints with 3 dimensions each.

Generating overlapping sequences...
Using (x,y)-centering...
Using all joints...

Generated 38,299 sequences of shape: Data(edge_attr=[5618], edge_index=[2, 5618], x=[53, 30], y=[53, 30])


# Construct model & train

In [147]:
# from torch_geometric.nn import NNConv
from functions.modules import MLP, MLPGraphConv

In [148]:
in_channels = data.seq_len*data.n_dim
out_channels = 30 # latent dimension
model = MLPGraphConv(in_channels=in_channels, out_channels=out_channels, 
                     nn=MLP(data[0].num_edge_features, 100, 2*in_channels*out_channels))
print(model)

MLPGraphConv(30, 30)


In [149]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print("Using {}".format(device))
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

Using cpu


In [158]:
loss_func = nn.MSELoss()

def train():
    losses = []
    model.train()
    for batch in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        output = model(batch.x, batch.edge_index, batch.edge_attr)
        loss = loss_func(output, batch.y)
        print("Loss: {}".format(loss.item()))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return losses

In [159]:
for epoch in range(10):
    train()

  0%|          | 0/19150 [00:00<?, ?it/s]

Loss: nan


  0%|          | 1/19150 [00:03<19:17:30,  3.63s/it]

Loss: nan


  0%|          | 2/19150 [00:07<19:24:28,  3.65s/it]

...

Loss: nan


  0%|          | 5/19150 [00:18<19:45:57,  3.72s/it]

Loss: nan


  0%|          | 6/19150 [00:23<20:56:24,  3.94s/it]


KeyboardInterrupt: 

# Scratch work

In [ ]:
# class MLPEncoder(MessagePassing):
#     def __init__(self, n_timesteps, n_joints, n_dim, hidden_dim, latent_dim, edge_features):
#         super(MLPEncoder, self).__init__(aggr='add')
#         self.mlp = MLP(n_joints*n_dim, hidden_dim, latent_dim)

#     def forward(self, x, edge_index):
#         # x has shape [n_timesteps, n_joints*n_dim]
#         # edge_index has shape [2, edge_features=1]
#         return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

#     def message(self, x_i, x_j):
#         # x_i has shape [edge_features, n_joints*n_dim]
#         # x_j has shape [edge_features, n_joints*n_dim]
#         tmp = torch.cat([x_i, x_j - x_i], dim=1)  # tmp has shape [edge_features=1, 2*n_joints*n_dim]
#         return self.mlp(tmp)

#     def update(self, aggr_out):
#         # aggr_out has shape [n_timesteps, latent_dim]
#         return aggr_out

In [ ]:
# import torch
# from torch.nn import Sequential as Seq, Linear, ReLU
# from torch_geometric.nn import MessagePassing

# class MLP_Encoder(nn.Module):
    
#     def __init__(self, in_dim_channel, out_dim_channel, hidden_dim):
#         super(MLP_Encoder, self).__init__()
        
# #         self.mlp1 = MLP(in_dim, hidden_dim, out_dim)
#         self.pass_mlp = MLP(in_dim, hidden_dim, out_dim)
#         self.pass1 = NNConv(in_dim_channel, out_dim_channel, self.pass_mlp)
# #         self.mlp2 = MLP(in_dim, hidden_dim, out_dim)
    
#     def forward(self, inputs : Data):
#         x = self.pass1(inputs.x, inputs.edge_index, inputs.edge_attr)
#         node_features = self.mlp2(inputs.x)
#         return x